# This Notebook will be used to train and test a CNN-Model that will be built from scratch with the intention of not using pre-trained models
## Author: MrHaso 2022 

### Imports

In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub 
from tensorflow.keras.layers import MaxPooling2D, Input, Conv2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.models import Model

### Create Train/Test sets

#### Create Array of Images

In [ ]:
chords = ["No_Chord","Am","B","C","D","Em","F","G"]
X = []
for folder in chords:
    output_path = "Images/class_"+folder+"/"
    for path in os.listdir(output_path):
        if os.path.isfile(os.path.join(output_path, path)):
            im = cv2.imread(os.path.join(output_path, path))
            X.append(im)
X = np.array(X)
print(X[:1])

#### Create Labels

In [ ]:
chords = ["No_Chord","Am","B","C","D","Em","F","G"]
labels = []
for folder in chords:
    output_path = "Images/class_"+folder+"/"
    for path in os.listdir(output_path):
        if os.path.isfile(os.path.join(output_path, path)):
            labels.append(folder)

y = pd.DataFrame(labels)
y = pd.factorize(y[0])[0].astype(np.uint16)

#### Train / Test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X,y,
                                                    test_size=0.2,
                                                    random_state=42)

len(X_train), len(y_train), len(X_test), len(y_test)

In [ ]:
X_train.shape, y_train.shape

In [ ]:
K = len(set(y_train))
print(f"We have {K} classes")

In [ ]:
sample_im = 34
plt.imshow(X_test[sample_im]);
print(y_test[sample_im])

### Create CNN-Model

In [ ]:
def build_simple_CNN():
    i = Input(shape=X_train[0].shape)
    x = Conv2D(64, (3,3), strides=2, activation="relu")(i)
    x = Conv2D(128, (3,3), strides=2, activation="relu")(x)
    x = Conv2D(256, (3,3), strides=2, activation="relu")(x)
    x = Flatten()(x)
    x = Dropout(0.2)(x)
    x = Dense(2048, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(K, activation="softmax")(x)

    model = Model(i,x)

    model.compile(optimizer="adam",
                  loss="sparse_categorical_crossentropy",
                  metrics=["accuracy"])
    return model

In [ ]:
def build_complex_CNN():
    i = Input(shape=X_train[0].shape)
    x = Conv2D(64, (3,3), activation="relu", padding="same")(i)
    x = BatchNormalization()(x)
    x = Conv2D(64, (3,3), activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(128, (3,3), activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3,3), activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)
    x = Conv2D(256, (3,3), activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = Conv2D(256, (3,3), activation="relu", padding="same")(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2,2))(x)

    x = Flatten()(x)
    x = Dropout(0.2)(x)
    x = Dense(2048, activation="relu")(x)
    x = Dropout(0.2)(x)
    x = Dense(K, activation="softmax")(x)

    model = Model(i,x)
    model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])
    
    batch_size= 32
    data_generator = tf.keras.preprocessing.image.ImageDataGenerator(width_shift_range=0.1,
                                                                     height_shift_range=0.1,
                                                                     horizontal_flip=True)
    data_generator.fit(X_train)
    train_generator = data_generator.flow(X_train,y_train,batch_size)
    steps_per_epoch = X_train.shape[0] // batch_size
    
    return model, train_generator, steps_per_epoch
    

In [ ]:

path_checkpoint = "training_1/cp.ckpt"
directory_checkpoint = os.path.dirname(path_checkpoint)

save_model = tf.keras.callbacks.ModelCheckpoint(filepath=path_checkpoint,
                                                 save_weights_only=True,
                                                 verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor="val_accuracy",
                                                  patience=10)


In [ ]:
simple_model = build_simple_CNN()
simple_model.summary()

In [ ]:
complex_model, train_generator, steps_per_epoch = build_complex_CNN()

complex_model.summary()

### Train and Test the Model

In [ ]:
simple_r = simple_model.fit(X_train,y_train, validation_data=(X_test, y_test), epochs=50, callbacks=[save_model,early_stopping])

In [ ]:
complex_r = complex_model.fit(train_generator,validation_data=(X_test,y_test),steps_per_epoch=steps_per_epoch,epochs=50,callbacks=[early_stopping])

In [ ]:
plt.plot(simple_r.history["loss"],label="simple loss")
plt.plot(simple_r.history["val_loss"],label="simple val_loss")
plt.legend();

In [ ]:
plt.plot(simple_r.history["accuracy"],label="simple acc")
plt.plot(simple_r.history["val_accuracy"],label="simple val_acc")
plt.legend();

In [ ]:
plt.plot(complex_r.history["loss"],label="complex loss")
plt.plot(complex_r.history["val_loss"],label="complex val_loss")

plt.legend();

In [ ]:
plt.plot(complex_r.history["accuracy"],label="complex acc")
plt.plot(complex_r.history["val_accuracy"],label="comlpex val_acc")
plt.legend();

### Live Detection

In [ ]:
loaded_model = build_CNN()

loss_, accuracy_ = loaded_model.evaluate(X_test, y_test, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * accuracy_))

In [ ]:
loaded_model.load_weights(path_checkpoint)

loss_, accuracy_ = loaded_model.evaluate(X_test, y_test, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * accuracy_))

In [ ]:
!mkdir -p saved_model
loaded_model.save('saved_model/loaded_model')

In [ ]:
plt.imshow(X_test[24]);

In [ ]:
x = X_test[24]
y = np.expand_dims(x, axis=0)
predictions = complex_model.predict(y)
print(predictions)

In [ ]:
unique_labels=[0,1,2,3,4,5,6,7,8]

In [ ]:
index = 24
print(predictions)
print(f"Max value (probability of prediction): {np.max(predictions)}")
print(f"Sum: {np.sum(predictions)}")
print(f"Max index: {np.argmax(predictions)}")
print(f"Predicted label: {unique_labels[np.argmax(predictions)]}")
print(f"Correct label: {y_test[index]}")

In [ ]:
chords = ["No_Chord","Am","B","C","D","Em","F","G"]

# Setup camera attributes to take pictures with HD-USB Cam 
cam = cv2.VideoCapture(0, cv2.CAP_DSHOW)
cam.set(10,150)
cam.set(cv2.CAP_PROP_FPS, 30.0)
cam.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('m','j','p','g'))
cam.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M','J','P','G'))
cam.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cam.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

size=(64,64)

while True:

    ret, frame = cam.read()
    frame = frame[0:120,0:120]
    
    frame2 = cv2.resize(frame,size)
    frame = cv2.flip(frame,1)
    frame2 = np.expand_dims(frame2, axis=0)
    frame3 = cv2.resize(frame,(480,480))
    res = simple_model.predict(frame2)
    cv2.putText(frame3,chords[np.argmax(res)],(20,50), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 8, cv2.LINE_AA)
    

    cv2.imshow("CAM",frame3)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    
cam.release()
cv2.destroyAllWindows()